In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from torch.autograd import Variable

train_x = Variable(torch.zeros(64, 2))
for i in range(8):
    for j in range(8):
        train_x.data[i * 8 + j][0] = float(i) / 8

train_y = Variable(torch.sign(torch.cos(train_x.data[:, 0] * (2 * math.pi)) * torch.cos(train_x.data[:, 1] * (2 * math.pi))))

In [3]:
from torch import nn, optim
from gpytorch.kernels import RBFKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood, BernoulliLikelihood
from gpytorch.random_variables import GaussianRandomVariable

In [4]:
class GPClassificationModel(gpytorch.GPModel):
    def __init__(self):
        super(GPClassificationModel,self).__init__(BernoulliLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = RBFKernel(log_lengthscale_bounds=(-6, 6))
        self.grid_covar_module = GridInterpolationKernel(self.covar_module)
        self.register_parameter('log_outputscale', nn.Parameter(torch.Tensor([0])), bounds=(-6,6))
        self.initialize_interpolation_grid(10)
    
    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.grid_covar_module(x)
        covar_x = covar_x.mul(self.log_outputscale.exp())
        latent_pred = GaussianRandomVariable(mean_x, covar_x)
        return latent_pred

prior_model = GPClassificationModel()

In [5]:
# def plot_model_and_predictions(model, plot_train_data=True):
#     f, observed_ax = plt.subplots(1, 1, figsize=(4, 3))
#     test_x = Variable(torch.zeros(1024, 2))
#     for i in range(32):
#         for j in range(32):
#             test_x.data[i * 32 + j][0] = float(i) / 32
#             test_x.data[i * 32 + j][1] = float(j) / 32
#     observed_pred = model(test_x)

#     def ax_plot(ax, rand_var, title):
#         if plot_train_data:
#             ax.plot(train_x.data.numpy(), train_y.data.numpy(), 'k*')
#         pred_labels = rand_var.mean().ge(0.5).float().mul(2).sub(1)
#         ax.plot(test_x.data.numpy(), pred_labels.data.numpy(), 'b')
#         ax.set_ylim([-3, 3])
#         ax.legend(['Observed Data', 'Mean', 'Confidence'])
#         ax.set_title(title)
    
#     ax_plot(observed_ax, observed_pred, 'Observed Values (Likelihood)')
    
#     return f

In [6]:
from gpytorch.inference import Inference
infer = Inference(prior_model)
posterior_model = infer.run(train_x, train_y)

# Find optimal model hyperparameters
posterior_model.train()
optimizer = optim.Adam(posterior_model.parameters(), lr=0.1)
optimizer.n_iter = 0
for i in range(200):
    optimizer.zero_grad()
    output = posterior_model.forward(train_x)
    loss = -posterior_model.marginal_log_likelihood(output, train_y)
    loss.backward()
    optimizer.n_iter += 1
    print('Iter %d/20 - Loss: %.3f   log_lengthscale: %.3f' % (
        i + 1, loss.data[0],
        posterior_model.prior_model.covar_module.log_lengthscale.data.squeeze()[0],
    ))
    optimizer.step()
    
# Set back to eval mode
posterior_model.eval()

(Variable containing:
-0.1429
 0.0000
 0.1429
 0.2857
 0.4286
 0.5714
 0.7143
 0.8571
 1.0000
 1.1429
[torch.FloatTensor of size 10]
,)


RuntimeError: size mismatch, m1: [10 x 10], m2: [100 x 10] at /Users/soumith/miniconda2/conda-bld/pytorch_1501999754274/work/torch/lib/TH/generic/THTensorMath.c:1293